In [1]:
############Imports##########
import sys
sys.path.append("/Users/michael.rowe/Personal/IntrinsicValueProject")
from createDS import dataset
from make_model_pipeline import make_model
from test_pipeline import tester
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

2023-10-30 03:26:21.036191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
###########Create Dataset########
asml = dataset('ASML','6mo','OG') #stock, period, training data
X_train, X_test, y_train, y_test, todays_data = asml.split(.8) #what train test split

In [14]:
##########Define model hyperparameters#########
optimizer = "adam"
loss = "mean_squared_error"
epochs = 2400
batch_size = 8

stock = asml.ticker
period = asml.period
train_data_name = asml.indicators_keyword
train_data = list(X_train.columns.values)
start_data = str(X_train.iloc[0]).split("Name:")[-1].split(", ")[0]
end_data = str(X_train.iloc[-1]).split("Name:")[-1].split(", ")[0]
num_train_points = len(X_train)

model_name = "LSTM2"

In [16]:
##########Model############
model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1],1)))
model.add(Dropout(rate = 0.2))

##add 2nd lstm layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(rate = 0.2))
##add 3rd lstm layer
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(rate = 0.2))
##add 4th lstm layer
model.add(LSTM(units = 50, return_sequences = False))
model.add(Dropout(rate = 0.2))

model.add(Dense(units = 1))

model.compile(optimizer = optimizer, loss = loss)

# Give a summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 10, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 10, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 10, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 10, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                2

In [17]:
##########Train model###########
model.fit(x = X_train, y = y_train, batch_size = batch_size, epochs = epochs)

Epoch 1/2400
10/10 [==============================] - 9s 22ms/step - loss: 0.0884
Epoch 2/2400
10/10 [==============================] - 0s 20ms/step - loss: 0.0866
Epoch 3/2400
10/10 [==============================] - 0s 20ms/step - loss: 0.0866
Epoch 4/2400
10/10 [==============================] - 0s 21ms/step - loss: 0.0879
Epoch 5/2400
10/10 [==============================] - 0s 21ms/step - loss: 0.0875
Epoch 6/2400
10/10 [==============================] - 0s 20ms/step - loss: 0.0861
Epoch 7/2400
10/10 [==============================] - 0s 19ms/step - loss: 0.0864
Epoch 8/2400
10/10 [==============================] - 0s 20ms/step - loss: 0.0859
Epoch 9/2400
10/10 [==============================] - 0s 22ms/step - loss: 0.0855
Epoch 10/2400
10/10 [==============================] - 0s 24ms/step - loss: 0.0843
Epoch 11/2400
10/10 [==============================] - 0s 20ms/step - loss: 0.0860
Epoch 12/2400
10/10 [==============================] - 0s 28ms/step - loss: 0.0851
Epoch 13/2400

In [ ]:
###############Testing pipeline############
X_preds = model.predict(X_test)
my_tester1 = tester(y_test,X_preds,model,X_test)
my_tester1.compare()
my_tester1.plot()

In [ ]:
##############Incremental Updating###########
#Might want to do this after saving the model
incremental_predictions,y_actuals = my_tester1.incremental_updating()
my_tester2 = tester(y_actuals,incremental_predictions,model,X_test)

my_tester2.compare()
my_tester2.plot()



In [18]:
###########Save Model###########
pipeline = make_model(model_name,model,optimizer,loss,epochs,batch_size,stock,period,train_data_name,train_data,start_data,end_data,num_train_points)
pipeline.save_model()

In [19]:
##########Create Record#########
pipeline.create_record()